# Understanding executables

Running on a laptop with an i7-9750H processor

* https://stackoverflow.com/questions/200292/process-for-reducing-the-size-of-an-executable
* http://www.muppetlabs.com/~breadbox/software/tiny/
* https://stac47.github.io/c/relocation/elf/tutorial/2018/03/01/understanding-relocation-elf.html
* http://timelessname.com/elfbin/

In [1]:
!lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         39 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  12
  On-line CPU(s) list:   0-11
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
    CPU family:          6
    Model:               158
    Thread(s) per core:  2
    Core(s) per socket:  6
    Socket(s):           1
    Stepping:            10
    CPU max MHz:         4500,0000
    CPU min MHz:         800,0000
    BogoMIPS:            5199.98
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mc
                         a cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss 
                         ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc art
                          arch_perfmon pebs bts rep_good nopl xtopology nonstop_
                         tsc cpuid aperfmperf pni pclmulqdq dtes64 monitor ds

In [5]:
%%writefile helloworld.c
#include <stdio.h>
extern int puts(const char *);
void main(void)
{
    puts("Hello, world!");
}

Overwriting helloworld.c


In [6]:
! gcc helloworld.c

In [7]:
! ls -gG a.out

-rwxrwxr-x 1 16704 set 10 20:34 a.out


In [8]:
! ./a.out

Hello, world!


-s  Remove all symbol table and relocation information from the executable.

In [9]:
! gcc -s helloworld.c 

In [10]:
! ls -gG a.out

-rwxrwxr-x 1 14472 set 10 20:38 a.out


-nostartfiles    Do not use the standard system startup files when linking. The standard libraries are used normally. 

In [12]:
! gcc -s -nostartfiles helloworld.c 

/usr/bin/ld: warning: cannot find entry symbol _start; defaulting to 0000000000001030


In [13]:
! ls -gG a.out

-rwxrwxr-x 1 13808 set 10 20:40 a.out


In [14]:
%%writefile tiny.c
int main(void)
{
    return 42;
}

Writing tiny.c


In [18]:
! gcc tiny.c

In [19]:
! ./a.out ; echo $?  # $? = last command return

42


In [20]:
! ls -gG a.out

-rwxrwxr-x 1 16464 set 10 20:45 a.out


In [21]:
! wc -c a.out

16464 a.out


In [22]:
! gcc -s -nostartfiles -nostdlib tiny.c

/usr/bin/ld: warning: cannot find entry symbol _start; defaulting to 0000000000001000


In [23]:
! ls -gG a.out

-rwxrwxr-x 1 13296 set 10 20:46 a.out


In [24]:
! objdump -x a.out


a.out:     file format elf64-x86-64
a.out
architecture: i386:x86-64, flags 0x00000150:
HAS_SYMS, DYNAMIC, D_PAGED
start address 0x0000000000001000

Program Header:
    PHDR off    0x0000000000000040 vaddr 0x0000000000000040 paddr 0x0000000000000040 align 2**3
         filesz 0x00000000000002d8 memsz 0x00000000000002d8 flags r--
  INTERP off    0x0000000000000318 vaddr 0x0000000000000318 paddr 0x0000000000000318 align 2**0
         filesz 0x000000000000001c memsz 0x000000000000001c flags r--
    LOAD off    0x0000000000000000 vaddr 0x0000000000000000 paddr 0x0000000000000000 align 2**12
         filesz 0x00000000000003b9 memsz 0x00000000000003b9 flags r--
    LOAD off    0x0000000000001000 vaddr 0x0000000000001000 paddr 0x0000000000001000 align 2**12
         filesz 0x000000000000000f memsz 0x000000000000000f flags r-x
    LOAD off    0x0000000000002000 vaddr 0x0000000000002000 paddr 0x0000000000002000 align 2**12
         filesz 0x0000000000000050 memsz 0x0000000000000050 flags r--
  

In [25]:
! hexdump -C a.out

00000000  7f 45 4c 46 02 01 01 00  00 00 00 00 00 00 00 00  |.ELF............|
00000010  03 00 3e 00 01 00 00 00  00 10 00 00 00 00 00 00  |..>.............|
00000020  40 00 00 00 00 00 00 00  b0 30 00 00 00 00 00 00  |@........0......|
00000030  00 00 00 00 40 00 38 00  0d 00 40 00 0d 00 0c 00  |....@.8...@.....|
00000040  06 00 00 00 04 00 00 00  40 00 00 00 00 00 00 00  |........@.......|
00000050  40 00 00 00 00 00 00 00  40 00 00 00 00 00 00 00  |@.......@.......|
00000060  d8 02 00 00 00 00 00 00  d8 02 00 00 00 00 00 00  |................|
00000070  08 00 00 00 00 00 00 00  03 00 00 00 04 00 00 00  |................|
00000080  18 03 00 00 00 00 00 00  18 03 00 00 00 00 00 00  |................|
00000090  18 03 00 00 00 00 00 00  1c 00 00 00 00 00 00 00  |................|
000000a0  1c 00 00 00 00 00 00 00  01 00 00 00 00 00 00 00  |................|
000000b0  01 00 00 00 04 00 00 00  00 00 00 00 00 00 00 00  |................|
000000c0  00 00 00 00 00 00 00 00  00 00 00 00 00 00

In [31]:
! strip --strip-all --verbose a.out

copy from `a.out' [elf64-x86-64] to `staUnA4g' [elf64-x86-64]


In [30]:
! ls -gG a.out

-rwxrwxr-x 1 13296 set 10 20:56 a.out


In [41]:
%%bash
gcc tiny.c -Wl,--gc-sections -fno-ident -Qn -s -nostartfiles -nostdlib  \
    -ffunction-sections -fdata-sections  \

/usr/bin/ld: warning: cannot find entry symbol _start; not setting start address


In [42]:
! ls -gG a.out

-rwxrwxr-x 1 9016 set 10 21:15 a.out


In [48]:
! strip a.out --remove-section=.comment --remove-section=.note* \
    --remove-section=.gnu.hash

In [49]:
! ls -gG a.out

-rwxrwxr-x 1 8848 set 10 21:21 a.out


In [51]:
! objdump -x a.out


a.out:     file format elf64-x86-64
a.out
architecture: i386:x86-64, flags 0x00000150:
HAS_SYMS, DYNAMIC, D_PAGED
start address 0x0000000000000000

Program Header:
    PHDR off    0x0000000000000040 vaddr 0x0000000000000040 paddr 0x0000000000000040 align 2**3
         filesz 0x0000000000000230 memsz 0x0000000000000230 flags r--
  INTERP off    0x00000000000002a8 vaddr 0x00000000000002a8 paddr 0x00000000000002a8 align 2**0
         filesz 0x000000000000001c memsz 0x000000000000001c flags r--
    LOAD off    0x0000000000000000 vaddr 0x0000000000000000 paddr 0x0000000000000000 align 2**12
         filesz 0x0000000000001010 memsz 0x0000000000001010 flags r--
    LOAD off    0x0000000000001f20 vaddr 0x0000000000002f20 paddr 0x0000000000002f20 align 2**12
         filesz 0x00000000000000e0 memsz 0x00000000000000e0 flags rw-
 DYNAMIC off    0x0000000000001f20 vaddr 0x0000000000002f20 paddr 0x0000000000002f20 align 2**3
         filesz 0x00000000000000e0 memsz 0x00000000000000e0 flags rw-
   

In [68]:
! gcc helloworld.c

In [69]:
! objdump -d a.out


a.out:     file format elf64-x86-64


Disassembly of section .init:

0000000000001000 <_init>:
    1000:	f3 0f 1e fa          	endbr64 
    1004:	48 83 ec 08          	sub    $0x8,%rsp
    1008:	48 8b 05 d9 2f 00 00 	mov    0x2fd9(%rip),%rax        # 3fe8 <__gmon_start__>
    100f:	48 85 c0             	test   %rax,%rax
    1012:	74 02                	je     1016 <_init+0x16>
    1014:	ff d0                	callq  *%rax
    1016:	48 83 c4 08          	add    $0x8,%rsp
    101a:	c3                   	retq   

Disassembly of section .plt:

0000000000001020 <.plt>:
    1020:	ff 35 9a 2f 00 00    	pushq  0x2f9a(%rip)        # 3fc0 <_GLOBAL_OFFSET_TABLE_+0x8>
    1026:	f2 ff 25 9b 2f 00 00 	bnd jmpq *0x2f9b(%rip)        # 3fc8 <_GLOBAL_OFFSET_TABLE_+0x10>
    102d:	0f 1f 00             	nopl   (%rax)
    1030:	f3 0f 1e fa          	endbr64 
    1034:	68 00 00 00 00       	pushq  $0x0
    1039:	f2 e9 e1 ff ff ff    	bnd jmpq 1020 <.plt>
    103f:	90                   	nop

Disassembly of se

In [2]:
! size a.out    # text = code

   text	   data	    bss	    dec	    hex	filename
   1551	    600	      8	   2159	    86f	a.out


In [3]:
! nm a.out    # symbols

0000000000004010 B __bss_start
0000000000004010 b completed.8060
                 w __cxa_finalize@@GLIBC_2.2.5
0000000000004000 D __data_start
0000000000004000 W data_start
0000000000001090 t deregister_tm_clones
0000000000001100 t __do_global_dtors_aux
0000000000003dc0 d __do_global_dtors_aux_fini_array_entry
0000000000004008 D __dso_handle
0000000000003dc8 d _DYNAMIC
0000000000004010 D _edata
0000000000004018 B _end
00000000000011d8 T _fini
0000000000001140 t frame_dummy
0000000000003db8 d __frame_dummy_init_array_entry
000000000000215c r __FRAME_END__
0000000000003fb8 d _GLOBAL_OFFSET_TABLE_
                 w __gmon_start__
0000000000002014 r __GNU_EH_FRAME_HDR
0000000000001000 t _init
0000000000003dc0 d __init_array_end
0000000000003db8 d __init_array_start
0000000000002000 R _IO_stdin_used
                 w _ITM_deregisterTMCloneTable
                 w _ITM_registerTMCloneTable
00000000000011d0 T __libc_csu_fini
0000000000001160 T __libc_csu_init
                 U __libc_star